**0) Lý thuyết và nguồn dữ liệu** <br>
Nguồn dữ liệu được lấy từ trang web: https://github.com/stefan-it/nmt-en-vi/tree/master/data

Lý thuyết tham khảo ở các tài liệu:


1.   Cho, K., Van Merriënboer, B., Bahdanau, D., & Bengio, Y. (2014). On the properties of neural machine translation: Encoder-decoder approaches. arXiv preprint arXiv:1409.1259.
2.   Rico Sennrich, Barry Haddow (2016). Linguistic Input Features Improve Neural Machine Translation.



**1) Chuẩn bị thư viện và dữ liệu**

Đầu tiên ta sẽ chuẩn bị một số thư viện

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
!pip install unidecode
import unidecode
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     |████████████████████████████████| 245kB 4.7MB/s 


Chuẩn bị dữ liệu cho chương trình

In [0]:
!git clone https://github.com/nguyenduyhanlam/PTXS-GTNN
!ls
import os
os.getcwd()

Cloning into 'PTXS-GTNN'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 3), reused 19 (delta 2), pack-reused 0
Unpacking objects: 100% (20/20), done.
PTXS-GTNN  sample_data


'/content'

**2) Load dữ liệu** <br>
Ta sẽ tạo class ngôn ngữ dùng để load dữ liệu từ file text. <br>
Class ngôn ngữ gồm có các **thuộc tính** cơ bản sau:
*   **name**: tên của ngôn ngữ (English, tiếng Việt,...)
*   **word2index**: bộ từ điển.
*   **word2count**: bộ đếm từ.
*   **index2word**: bộ lưu vị trí các token.
*   **n_words**: đếm tổng các lượng từ.

Class ngôn ngữ gồm có các **hàm** cơ bản sau:
*   **addSentence**: Có tham số là sentence, dùng để truyền vào 1 câu văn bản. Từ câu văn bản này, ta sẽ tách thành các từ (word). Cuối cùng ta sẽ lưu các từ này vào bộ từ điển bằng hàm addWord.
*   **addWord**: Có tham số là word, dùng để truyền vào 1 từ (word), nếu từ này chưa có trong bộ từ điển thì sẽ được thêm (add) mới vào bộ từ điển. Nếu đã có rồi ta sẽ tăng số lần xuất hiện (số đếm) của từ này thêm 1 đơn vị.








In [0]:
SOS_token = 0 #Start Of Sequence
EOS_token = 1 #End Of Sequence
PAD_token = 2 # Used for padding short sentences

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Ta cần triển khai thêm một số hàm bổ trợ như:
*   **remove_accent**: dùng để loại bỏ các dấu thanh (sắc, hỏi, huyền, ngã, nặng,...).
*   **normalizeString**: Dùng để chuẩn hóa câu để chạy chương trình. Đầu tiên ta sẽ loại bỏ hết các dấu thanh ra khỏi câu. Sau đó ta sẽ tách các dấu câu (dấu phẩy, dấu chấm, dấu chấm hỏi, dấu chấm thang,...) ra khỏi từ.

Ví dụ ta có câu: "Ăn quả, nhớ kẻ trồng cây.". Sau khi chạy hàm **normalizeString** ta sẽ được kết quả là: "An qua , nho ke trong cay ."



In [0]:
# remove all the accents
def remove_accent(utf8_str):
    return unidecode.unidecode(utf8_str)

# Normalize the string (marks and words are seperated, words don't contain accents,...)
def normalizeString(s):
    # Remove all the accents first.
    s = remove_accent(s)
    # Seperate words and marks by adding spaces between them
    marks = '[.!?,-${}()]'
    r = "(["+"\\".join(marks)+"])"
    s = re.sub(r, r" \1 ", s)
    # replace continuous spaces with a single space
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Example
ex_s = "Ăn quả, nhớ kẻ trồng cây."
print(normalizeString(ex_s)) # result will be "An qua , nho ke trong cay ."

An qua , nho ke trong cay .


Cuối cùng ta sẽ thực hiện việc load dữ liệu.

In [0]:
def readLangs(lang1, lang2):
    print("Reading lines...")

    # Read the file and split into lines
    lines_language1 = open('PTXS-GTNN/tst2013.%s' % lang1, encoding='utf-8').\
        read().strip().split('\n')
    lines_language2 = open('PTXS-GTNN/tst2013.%s' % lang2, encoding='utf-8').\
        read().strip().split('\n')

    # Normalize all the lines
    data_language1 = [normalizeString(l) for l in lines_language1]
    data_language2 = [normalizeString(l) for l in lines_language2]

    # Prepare return values
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    data = list(zip(data_language1, data_language2))

    return input_lang, output_lang, data

# Test the function
lang1 = "en"
lang2 = "vi"
input_lang, output_lang, data = readLangs(lang1, lang2)
print("Language 1:", input_lang.name)
print("Language 2:", output_lang.name)
print(random.choice(data))

Reading lines...
Language 1: en
Language 2: vi
('In fact , they &apos;re often the initial and convincing negotiators of a bright future for their daughters , but in the context of a society like in Afghanistan , we must have the support of men .', 'That ra , ho la nguoi dau tien va la nguoi thuong thuyet thuyet phuc cho mot tuong lai sang cua con gai ho , nhung trong hoan canh xa hoi nhu Afghanistan , chung ta can den su ung ho cua nhung nguoi dan ong .')


**3) Chuẩn bị dữ liệu** <br>


Chương trình sẽ chạy nhanh hơn khi chúng ta cắt bộ dữ liệu thành các câu ngắn và đơn giản. Ở đây ta sẽ giới hạn cho 1 câu có tối đa là 10 từ (bao gồm cả dấu chấm câu).


In [0]:
MAX_LENGTH = 10

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Sau quá trình biến đổi chúng ta sẽ có được tập dữ liệu để chuẩn bị cho huấn luyện.

In [0]:
def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print("Language 1:", input_lang.name, "There are", input_lang.n_words, "different words")
    print("Language 2:", output_lang.name, "There are", output_lang.n_words, "different words")
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('en', 'vi')
print(random.choice(pairs))

Reading lines...
Read 1268 sentence pairs
Trimmed to 135 sentence pairs
Counting words...
Counted words:
Language 1: en There are 320 different words
Language 2: vi There are 334 different words
('Let &apos;s see . Yeah , it worked .', 'Xem ne . Vang , no dinh .')


**4) Mô hình Encoder-Decoder**

Đầu tiên ta sẽ đi xây dưng bộ encoder trước.

---

===================================**Encoder**===================================

Mô hình encoder như sau:

![Encoder example](https://drive.google.com/uc?id=1cyor63sz5r_-hnwXvptVDl_4-3ltsKzC)

(Source: https://medium.com/@t.schnake/a-formalization-of-a-simple-sequential-encoder-decoder-b31be7e92988)

![Pytorch encoder flow](https://drive.google.com/uc?id=1R5dtbS_Xl4wYZMJcL4BT47VEGX-lMMXR)

(Source: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

Ta sẽ đi xây dựng một class có tên là EncoderRNN cho bộ GRU-RNN encoder.

Class này có các thuộc tính cơ bản sau:
*   **hidden_size**: kích thước (số chiều) của vector của mỗi phần tử trong bộ từ điển.
*   **embedding**: lưu trữ kết quả của bộ từ điển sau khi thực hiện word embedding (tham khảo thêm về word embedding tại https://machinelearningmastery.com/what-are-word-embeddings/).
*   **gru**: cấu trúc GRU sẽ sử dụng.

Trong hàm khởi tạo ta sẽ truyền vào các biến sau:
*   **input_size**: kích thước của bộ từ điển (số lượng từ có trong bộ từ điển).
*   **hidden_size**: kích thước (số chiều) của vector của mỗi phần tử trong bộ từ điển.












In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        # A simple lookup table that stores embeddings of a fixed dictionary 
        # and size. 
        # This module is often used to store word embeddings and retrieve them using indices. 
        # The input to the module is a list of indices, 
        # and the output is the corresponding word embeddings.
        # input_size: num_embeddings (python:int) – size of the dictionary of embeddings
        # hidden_size: embedding_dim (python:int) – the size of each embedding vector
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        # Applies a multi-layer gated recurrent unit (GRU) RNN to an input sequence.
        # hidden_size: input_size – The number of expected features in the input x
        # hidden_size: hidden_size – The number of features in the hidden state h
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # Create word embedding table based on the data -> tensor table
        # view(*shape): Returns a new tensor with the same data as the self tensor but of a different shape.
        embedded = self.embedding(input).view(1, 1, -1)

        output = embedded

        # Execute GRU process
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        # Initiate the first hidden unit
        return torch.zeros(1, 1, self.hidden_size, device=device)

===================================**Decoder**===================================

Mô hình decoder cơ bản có thể được minh họa như sau:

![Decoder example](https://drive.google.com/uc?id=15k-FuqT8oCaP3MkHV7W-5OQg072OzQsz)

(Source: https://medium.com/@t.schnake/a-formalization-of-a-simple-sequential-encoder-decoder-b31be7e92988)

![Pytorch base decoder flow](https://drive.google.com/uc?id=16Mt0xS53HsAczMiwZYbcBRC0x95wA_og)

(Source: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

Class này có các thuộc tính cơ bản sau:
*   **hidden_size**: kích thước (số chiều) của vector của mỗi phần tử trong tập kết quả của encoder.
*   **embedding**: lưu trữ kết quả của bộ vector kết quả từ encoder sau khi thực hiện word embedding (tham khảo thêm về word embedding tại https://machinelearningmastery.com/what-are-word-embeddings/).
*   **gru**: cấu trúc GRU sẽ sử dụng.
*   **out**: đối tượng linear transformation.
*   **softmax**: đối tượng softmax.

Trong hàm khởi tạo ta sẽ truyền vào các biến sau:
*   **hidden_size**: kích thước (số chiều) của vector của mỗi phần tử trong tập kết quả của encoder.
*   **output_size**: kích thước của kết quả đầu ra.

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # A simple lookup table that stores embeddings of a fixed dictionary 
        # and size. 
        # This module is often used to store word embeddings and retrieve them using indices. 
        # The input to the module is a list of indices, 
        # and the output is the corresponding word embeddings.
        # output_size: num_embeddings (python:int) – size of the dictionary of embeddings
        # hidden_size: embedding_dim (python:int) – the size of each embedding vector
        self.embedding = nn.Embedding(output_size, hidden_size)

        # Applies a multi-layer gated recurrent unit (GRU) RNN to an input sequence.
        # hidden_size: input_size – The number of expected features in the input x
        # hidden_size: hidden_size – The number of features in the hidden state h
        self.gru = nn.GRU(hidden_size, hidden_size)

        # Applies a linear transformation to the incoming data: y = xA^T + b
        # hidden_size: in_features – size of each input sample
        # output_size: out_features – size of each output sample
        self.out = nn.Linear(hidden_size, output_size)
        
        # Applies the log(Softmax(x) function to an n-dimensional input Tensor. The LogSoftmax formulation can be simplified as:
        # logSoftmax(xi) = log(exp(xi) / sum_j(exp(xj)))
        # dim=1: Input: (*) where * means, any number of additional dimensions
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        # Create word embedding table based on the data -> tensor table
        # view(*shape): Returns a new tensor with the same data as the self tensor but of a different shape.
        output = self.embedding(input).view(1, 1, -1)
        
        # Assuring the encoder output is not negative
        # Because ReLU output range is (0, +inf)
        output = F.relu(output)

        # Execute GRU process
        output, hidden = self.gru(output, hidden)

        # Calculate the softmax
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        # Initiate the first hidden unit
        return torch.zeros(1, 1, self.hidden_size, device=device)

**Decoder sử dụng attention**

Attention là một cơ chế kỹ thuật nhằm giúp giải quyết các giới hạn của kiến trúc encoder-decoder trên các chuỗi dài (chuỗi dữ liệu đầu vào). Attention sẽ giúp tăng tốc độ học tập và cải thiện khả năng dự đoán của mô hình.

Do vector ngữ cảnh này có độ dài cố định và là kết quả sau quá trình mã hóa từ dữ liệu đầu vào có độ dài thay đổi, nên vector này sẽ chứa rất nhiều thông tin. Việc mã hóa (decode) vector này sẽ là một áp lực lớn đối với decoder.

Cơ chế attention cho phép tại mỗi bước input của decoder, decoder có thể tập trung vào từng phần khác nhau của encoder output. Đầu tiên, chúng ta sẽ tính tập trọng số attention (attention weight). Sau đó ta sẽ lấy tập trọng số này nhân với các vectơ đầu ra của encoder để tạo ra một tổ hợp có trọng số. Kết quả (được gọi là attn_applied) này giúp nhận biết được từng phần của chuỗi đầu vào, và từ đó sẽ giúp decoder chọn được từ (word) đầu ra đúng.

Thay vì mã hóa chuỗi đầu vào thành một vectơ ngữ cảnh cố định duy nhất, mô hình chú ý phát triển một vectơ bối cảnh được lọc riêng cho từng bước thời gian t ở đầu ra.

![Attention](https://drive.google.com/uc?id=1uK8t6wEnWEfpC4osuankfgH7WMXrx4ec)

(Source: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)


In [0]:
print("Source: https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3")
from IPython.display import Image
Image(url='https://miro.medium.com/max/576/1*wBHsGZ-BdmTKS7b-BtkqFQ.gif')

Source: https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3


Ta sẽ đi xây dựng class decoder có sử dụng attention.

![alt text](https://drive.google.com/uc?id=1MZ4vaGKFX5Q8cd-Z40i-rYRHg2yh5pDd)

(Source: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

Class này có các thuộc tính cơ bản sau:
*   **hidden_size**: kích thước (số chiều) của vector của mỗi phần tử trong tập kết quả của encoder.
*   **output_size**: kích thước của đầu ra.
*   **dropout_p**: Trong quá trình đào tạo, gán ngẫu nhiên giá trị 0 cho một vài phần tử của decoder input với xác suất p theo phân phối Bernoulli (tiếng việt: https://maths.uel.edu.vn/Resources/Docs/SubDomain/maths/TaiLieuHocTap/ToanUngDung/phn_phi_bernoulli.html, tiếng anh: https://en.wikipedia.org/wiki/Bernoulli_distribution). Mỗi channel sẽ được 0 hóa độc lập trong mỗi lần forward (lan truyền tiến). Đây đã được chứng minh là một kỹ thuật hiệu quả để chuẩn hóa (regularization) và ngăn chặn sự đồng thích ứng (co-adaptation) của các nơ-ron thần kinh như được mô tả trong bài viết https://arxiv.org/pdf/1207.0580.pdf. Hơn nữa, các đầu ra được chia tỷ lệ theo hệ số 1 / (1-p) trong quá trình đào tạo. Điều này có nghĩa là trong quá trình đánh giá, mô-đun chỉ cần tính toán một hàm identity.
*   **max_length**: số lượng từ giới hạn cho 1 câu dịch.
*   **embedding**: lưu trữ kết quả của bộ vector kết quả từ encoder sau khi thực hiện word embedding (tham khảo thêm về word embedding tại https://machinelearningmastery.com/what-are-word-embeddings/).
*   **attn**: .
*   **attn_combine**: .
*   **dropout**: .
*   **gru**: cấu trúc GRU sẽ sử dụng.
*   **out**: đối tượng linear transformation.

Trong hàm khởi tạo ta sẽ truyền vào các biến sau:
*   **hidden_size**: kích thước (số chiều) của vector của mỗi phần tử trong tập kết quả của encoder.
*   **output_size**: kích thước của kết quả đầu ra.
*   **dropout_p**: .
*   **max_length**: .

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        # A simple lookup table that stores embeddings of a fixed dictionary 
        # and size. 
        # This module is often used to store word embeddings and retrieve them using indices. 
        # The input to the module is a list of indices, 
        # and the output is the corresponding word embeddings.
        # output_size: num_embeddings (python:int) – size of the dictionary of embeddings
        # hidden_size: embedding_dim (python:int) – the size of each embedding vector
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)

        # Applies a linear transformation to the incoming data: y = xA^T + b
        # hidden_size * 2: in_features – size of each input sample
        # max_length: out_features – size of each output sample
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)

        # Applies a linear transformation to the incoming data: y = xA^T + b
        # hidden_size * 2: in_features – size of each input sample
        # hidden_size: out_features – size of each output sample
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)

        # During training, randomly zeroes some of the elements of 
        # the input tensor with probability p 
        # using samples from a Bernoulli distribution. 
        # Each channel will be zeroed out independently on every forward call.
        # This has proven to be an effective technique for regularization 
        # and preventing the co-adaptation of neurons 
        # as described in the paper Improving neural networks by preventing co-adaptation of feature detectors .
        # Furthermore, the outputs are scaled by a factor of 1/(1-p) 
        # during training. This means that during evaluation the module simply computes an identity function.
        # dropout_p: p – probability of an element to be zeroed. Default (when don't have parameter): 0.5
        # In this constructor, the default parameter (probability) is 0.1
        self.dropout = nn.Dropout(self.dropout_p)

        # Applies a multi-layer gated recurrent unit (GRU) RNN to an input sequence.
        # hidden_size: input_size – The number of expected features in the input x
        # hidden_size: hidden_size – The number of features in the hidden state h
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        
        # Applies a linear transformation to the incoming data: y = xA^T + b
        # hidden_size: in_features – size of each input sample
        # output_size: out_features – size of each output sample
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        # Create word embedding table based on the data -> tensor table
        # view(*shape): Returns a new tensor with the same data as the self tensor but of a different shape.
        embedded = self.embedding(input).view(1, 1, -1)
        
        # Execute dropout process
        embedded = self.dropout(embedded)

        # Execute attention process
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        # Performs a batch matrix-matrix product
        # unsqueeze: Returns a new tensor with a dimension of size one inserted at the specified position.
        #            The returned tensor shares the same underlying data with this tensor.
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        # Assuring the encoder output is not negative
        # Because ReLU output range is (0, +inf)
        output = F.relu(output)

        # Execute GRU process
        output, hidden = self.gru(output, hidden)

        # Calculate the softmax
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        # Initiate the first hidden unit
        return torch.zeros(1, 1, self.hidden_size, device=device)

**5) Quá trình huấn luyện**

Để huấn luyện, đối với mỗi cặp, chúng ta sẽ cần một tensor (trong toán là vector) đầu vào (index của các từ trong câu) và tensor đích (index của các từ trong câu đích). Trong khi tạo các vector này, ta sẽ nối thêm mã thông báo EOS vào cả hai chuỗi.

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)